In [0]:
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv2D, Input, Lambda, Flatten, Reshape, MaxPooling2D, Multiply, Activation, Add, Concatenate
from keras.applications.vgg19 import VGG19 
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
import keras.backend as K
import tensorflow as tf
from keras.engine.topology import Layer
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
import h5py
import matplotlib.pyplot as plt
from google.colab import drive

Using TensorFlow backend.


In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
from tensorflow.python.client import device_lib
assert len(K.tensorflow_backend._get_available_gpus()) > 0
assert 'GPU' in str(device_lib.list_local_devices())

In [0]:
drive.mount('/content/drive', force_remount=True)

In [0]:
%cd "drive/My Drive/BTP/code"
%ls

/content/drive/My Drive/BTP/code
Attention_train_v1.ipynb       __init__.py
Attention_train_v2.ipynb       multi-head_attention_mnist_v1.ipynb
Attention_train_v3-LTPA.ipynb  multi-head_attention_mnist_v2.ipynb
Bateaux_code/                  multi-head_attention_mnist_v3.ipynb
bateaux_metrics_v1.py          playground.ipynb
bateaux_metrics_v2.py          __pycache__/
dataset_creation/              siamese.ipynb
evaluate_models.ipynb          Untitled0.ipynb
extra/


In [0]:
from bateaux_metrics_v2 import *

In [0]:
parent_dir = os.path.dirname(os.getcwd())
train_set_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'train_set.hdf5')
train_GT_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'train_GT.csv')
test_set_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'test_set.hdf5')
test_GT_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'test_GT.csv')

weight_file = os.path.join(parent_dir, 'models', 'Attention', '1-scene', 'att_model-LTPA_v1.hdf5')
info_file = os.path.join(parent_dir, 'models', 'Attention', '1-scene', 'att_model-LTPA_v1.txt')

In [0]:
img_width = 128
img_height = 128
channels = 3

In [0]:
K.clear_session()

In [0]:
def vgg_block():
    #x = Input(shape=(img_width, img_height, channels))
    base_model = VGG19(include_top=False, input_shape=(img_width, img_height, channels))
    for layer in base_model.layers[:-6]:
        layer.trainable = False

    outputs = [base_model.get_layer('block3_pool').output, base_model.get_layer('block4_conv3').output, base_model.get_layer('block5_conv1').output, base_model.get_layer('block5_pool').output]
    pre_trained = Model(input=base_model.input, output=outputs)
    return pre_trained

In [0]:
class ParametrisedCompatibility(Layer):
    def __init__(self, kernel_regularizer=None, **kwargs):
        super(ParametrisedCompatibility, self).__init__(**kwargs)
        self.regularizer = kernel_regularizer

    def build(self, input_shape):
        self.u = self.add_weight(name='u', shape=(input_shape[0][3], 1), initializer='uniform', regularizer=self.regularizer, trainable=True)
        super(ParametrisedCompatibility, self).build(input_shape)

    def call(self, x):  # add l and g. Dot the sum with u.
        return K.dot(K.map_fn(lambda lam: (lam[0]+lam[1]),elems=(x),dtype='float32'), self.u)

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], input_shape[0][2])

In [0]:
weight_decay=0.0005
regularizer = keras.regularizers.l2(weight_decay)

inp = Input((img_width, img_height, channels), name='input_layer')

vgg = vgg_block()
(local1, local2, local3, g) = vgg(inp)




80142336/80134624 [==============================] - 3s 0us/step




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  


In [0]:
print(local1.get_shape)
print(local2.get_shape)
print(local3.get_shape)
print(g.get_shape)

<bound method Tensor.get_shape of <tf.Tensor 'model_1/block3_pool/MaxPool:0' shape=(?, 16, 16, 256) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'model_1/block4_conv3/Relu:0' shape=(?, 16, 16, 512) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'model_1/block5_conv1/Relu:0' shape=(?, 8, 8, 512) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'model_1/block5_pool/MaxPool:0' shape=(?, 4, 4, 512) dtype=float32>>


In [0]:
g = Flatten()(g)
g = Dense(512, activation='relu')(g)
print(g.get_shape)

<bound method Tensor.get_shape of <tf.Tensor 'dense_1/Relu:0' shape=(?, 512) dtype=float32>>


In [0]:
l1 = Dense(512, kernel_regularizer=regularizer, name='l1connectordense')(local1)  # batch*x*y*512
c1 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc1')([l1, g])  # batch*x*y
flatc1 = Flatten(name='flatc1')(c1)  # batch*xy
a1 = Activation('softmax', name='softmax1')(flatc1)  # batch*xy

reshaped1 = Reshape((-1,512), name='reshape1')(l1)  # batch*xy*512.
g1 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g1')([a1, reshaped1])  # batch*512.

l2 = local2
c2 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc2')([l2, g])
flatc2 = Flatten(name='flatc2')(c2)
a2 = Activation('softmax', name='softmax2')(flatc2)

reshaped2 =  Reshape((-1,512), name='reshape2')(l2)
g2 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g2')([a2, reshaped2])

l3 = local3
c3 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc3')([l3, g])
flatc3 = Flatten(name='flatc3')(c3)
a3 = Activation('softmax', name='softmax3')(flatc3)

reshaped3 = Reshape((-1,512), name='reshape3')(l3)
g3 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g3')([a3, reshaped3])
glist = [g3, g2, g1]
predictedG = g3
predictedG = Concatenate(axis=1, name='ConcatG')(glist)
out = Dense(6, kernel_regularizer=regularizer, name='output')(predictedG)
model = Model(inputs=inp, outputs=out)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 [(None, 16, 16, 256) 20024384    input_layer[0][0]                
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 8192)         0           model_1[1][3]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          4194816     flatten_1[0][0]                  
____________________________________________________________________________________________

In [0]:
# l2 = local2
# c2 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc2')([l2, g])
# flatc2 = Flatten(name='flatc2')(c2)
# a2 = Activation('softmax', name='softmax2')(flatc2)

# reshaped2 =  Reshape((-1,512), name='reshape2')(l2)
# g2 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g2')([a2, reshaped2])

In [0]:
# l3 = local3
# c3 = ParametrisedCompatibility(kernel_regularizer=regularizer, name='cpc3')([l3, g])
# flatc3 = Flatten(name='flatc3')(c3)
# a3 = Activation('softmax', name='softmax3')(flatc3)

# reshaped3 = Reshape((-1,512), name='reshape3')(l3)
# g3 = Lambda(lambda lam: K.squeeze(K.batch_dot(K.expand_dims(lam[0], 1), lam[1]), 1), name='g3')([a3, reshaped3])

In [0]:
# glist = [g3, g2, g1]
# predictedG = g3
# predictedG = Concatenate(axis=1, name='ConcatG')(glist)
# out = Dense(6, kernel_regularizer=regularizer, name='output')(predictedG)
# model = Model(inputs=inp, outputs=out)

In [0]:
epochs = 100
batch_size = 32
epochs_to_wait_for_improve = 3

In [0]:
lr = 0.005
momentum = 0
nesterov = False
#optimizer = SGD(lr=lr, momentum=momentum, nesterov=nesterov)
optimizer = Adam()
model.compile(optimizer=optimizer, loss=euclidean_loss(beta=beta), metrics=[euclidean_dist, x_error, y_error, z_error])
early_stopping_callback = EarlyStopping(monitor='val_euclidean_dist', patience=epochs_to_wait_for_improve, restore_best_weights=True)

In [0]:
h5_reader = h5py.File(train_set_file, 'r')
print(list(h5_reader.keys()))
y = np.loadtxt(train_GT_file, dtype=np.float32)
split_idx = int(y.shape[0] * 0.9)

x_train = np.array(h5_reader['train_img'][:split_idx], dtype=np.float32, copy=False) / 255
x_val = np.array(h5_reader['train_img'][split_idx:], dtype=np.float32, copy=False) / 255
y_train = y[:split_idx, :]
y_val = y[split_idx:, :]

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

h5_reader.close()

['train_img']
(23759, 128, 128, 3)
(23759, 6)
(2640, 128, 128, 3)
(2640, 6)


In [0]:
steps_per_epoch = x_train.shape[0] // batch_size
print(steps_per_epoch)

742


In [0]:
img_gen = ImageDataGenerator()

In [0]:
model.fit_generator(img_gen.flow(x_train, y_train, batch_size=batch_size),
            steps_per_epoch=steps_per_epoch, validation_data=(x_val, y_val),
            epochs=epochs, callbacks=[early_stopping_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
742/742 [==============================] - 160s 215ms/step - loss: 10.9675 - euclidean_dist: 2.3601 - x_error: 2.0864 - y_error: 2.0093 - z_error: 2.0147 - val_loss: 5.8453 - val_euclidean_dist: 1.4003 - val_x_error: 1.3476 - val_y_error: 1.6239 - val_z_error: 0.7655
Epoch 2/100
742/742 [==============================] - 149s 201ms/step - loss: 3.8664 - euclidean_dist: 0.7869 - x_error: 0.6937 - y_error: 0.7006 - z_error: 0.6486 - val_loss: 2.7152 - val_euclidean_dist: 0.6405 - val_x_error: 0.5393 - val_y_error: 0.7861 - val_z_error: 0.3651
Epoch 3/100
547/742 [=====================>........] - ETA: 36s - loss: 2.4841 - euclidean_dist: 0.4979 - x_error: 0.4051 - y_error: 0.4187 - z_error: 0.4628

In [0]:
model.save(weight_file)
print('Saved trained model at %s ' % weight_file)

In [0]:
h5_reader = h5py.File(test_set_file, 'r')
#print(h5_reader.keys())
x_test = h5_reader['test_img']
x_test = np.array(x_test)
h5_reader.close()
x_test = x_test.astype('float32')
x_test /= 255

In [0]:
y_test = np.loadtxt(test_GT_file, dtype=np.float32)

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
metric_names = model.metrics_names

In [0]:
for metric, val in zip(metric_names, scores):
  print("{} : {}".format(metric, val))

In [0]:
file_obj = open(info_file, 'w')
file_obj.write("MODEL SUMMARY\n")
#summary = model.summary()
model.summary(print_fn=lambda x: file_obj.write(x + '\n'))
file_obj.write("\n")
#file_obj.write("Optimizer : Adam - batch_size = {}\n\n".format(batch_size))
file_obj.write("Optimizer : SGD - lr = {}, momentum = {}, batch_size = {}\n\n".format(lr, momentum, batch_size))

file_obj.write("\n\nPerformance")
for metric, val in zip(metric_names, scores):
  file_obj.write("{} : {}\n".format(metric, val))

file_obj.write("\n\nFILE PATHS\n")
file_obj.write("Training set : {}\n".format(train_set_file))
file_obj.write("Testing set : {}\n".format(test_set_file))
file_obj.write("Weight file : {}\n".format(weight_file))
file_obj.close()